# Unsloth training 

This notebook uses the unsloth library for training the model to try to get faster iteration and less memory errors



In [ ]:
import torch
#the separation of the import and the installation may help avoid the torch issue I have been having, I don't know unlsoth reccomend
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
#second import because [[::shrug]]
from unsloth import FastLanguageModel
import torch
from datasets import DatasetDict
from trl import SFTTrainer
from transformers import TrainingArguments
import os
import wandb
from dotenv import load_dotenv

load_dotenv()

max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:


# start a new wandb run to track this script

api_key = os.getenv('WANDB_API_KEY')

wandb.login(api_key_or_path=api_key) 

In [ ]:

#We take the instruct model 
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
def create_prompt(sample):
    bos_token = "<s>"
    original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    system_message = "Use the provided input to create an instruction that could have been used to generate the response with an LLM."
    response = sample["prompt"].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
    input = sample["response"]
    eos_token = "</s>"

    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "### Instruction:"
    full_prompt += "\n" + system_message
    full_prompt += "\n\n### Input:"
    full_prompt += "\n" + input
    full_prompt += "\n\n### Response:"
    full_prompt += "\n" + response
    full_prompt += eos_token

    return { "text" : full_prompt, }

dataset = DatasetDict.load_from_disk('silver_dataset')
#convert dataset to form that can be fed directly to algo
dataset_mapped = dataset.map(create_prompt, batched = True,)

In [ ]:
training_args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=2,
        max_steps = None,
        learning_rate = 2e-4,
        report_to="wandb",  # enable logging to W&B
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    )

In [ ]:


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_mapped,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # will keep packing false for now may change later
    args = training_args,
)

## Train the model

In [ ]:
import time

wandb.init(
    # set the wandb project where this run will be logged
    project="mistral_victoria_test"
)

start = time.time()
trainer.train()
print(time.time()- start)

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()